In [2]:
import timm
import torch
import torch.nn as nn
import numpy as np

In [3]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('vit_base_patch16_224', pretrained=True)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [4]:
dc = model.named_children()
dc = list(dc)
x = torch.arange(4*3*224*224, dtype=torch.float32).reshape(4, 3, 224, 224)
x.shape

torch.Size([4, 3, 224, 224])

In [5]:
layer = 0
block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
# list(block['attn'].children())[0]
ls = [list(block['attn'].children())[0], list(block['attn'].children())[4],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]
ls[0]
# .weight.clone().detach().numpy().T

Linear(in_features=768, out_features=2304, bias=True)

## block binaryfile 생성기


In [6]:
for layer in range(12):
    block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
    ls = [block['norm1'],list(block['attn'].children())[0], list(block['attn'].children())[4],block['norm2'],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]

    weight_ls = list()
    for i in range(len(ls)):
        weight_ls.append(ls[i].weight.clone().detach().numpy().T)
        weight_ls.append(ls[i].bias.clone().detach().numpy())

    blk_weight = np.array(weight_ls[0].flatten())

    for i in range(1, len(weight_ls)):
        blk_weight = np.concatenate((blk_weight, weight_ls[i].flatten()))
    blk_weight.tofile('./pre_weights/'+str(layer)+'_newblock'+'.bin')

In [7]:
print(dc[0])
print("weight : ",dict(dc[0][1].named_children())['proj'].weight.reshape(768, 3, -1).shape)
print("\nx: ",x.shape)
print(dc[0][1](x).shape)

('patch_embed', PatchEmbed(
  (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (norm): Identity()
))
weight :  torch.Size([768, 3, 256])

x:  torch.Size([4, 3, 224, 224])
torch.Size([4, 196, 768])


### QKV WEIGHT BINFILE

In [8]:
layer = 0
dd = dict(list(dc[4][1].children())[layer].named_children())
blk = dict(dd['attn'].named_children())

In [9]:
torch.manual_seed(10)
dummy_input = torch.randn((4, 196, 768))
qkv_w = dict(dd['attn'].named_children())['qkv'].weight.clone().detach().T


blk['qkv'](dummy_input)[1]

tensor([[-0.2055, -1.8644, -1.5705,  ...,  0.5286,  0.3821, -0.2423],
        [ 1.3894, -0.5281, -1.4717,  ..., -0.2188, -0.8244, -0.4229],
        [ 0.8834, -0.5625, -0.1614,  ..., -0.5791, -0.1525, -0.2568],
        ...,
        [ 0.3980, -1.7263, -1.2302,  ..., -0.6791, -0.0356,  0.7446],
        [-0.2545, -2.0227, -1.4990,  ...,  0.3886,  0.2097, -0.2962],
        [ 1.8092, -0.4103, -0.1971,  ...,  0.2622,  0.4505,  0.0511]],
       grad_fn=<SelectBackward0>)

In [10]:
dummy_input.numpy().tofile('./pre_weights/dummy_input_4_196_768.bin')

In [11]:
dd_ = list(dd.values())
x = torch.arange(4 * 196 *768, dtype=torch.float32).reshape(4, 196, 768)
x = dd_[0](x)
qkv = list(dd_[1].children())
print(x.shape)
x = qkv[0](x)
print(x.shape)
x = qkv[1](x)
print(qkv[1],x.shape)
x = qkv[2](x)
print(qkv[2],x.shape)
x = qkv[3](x)
print(qkv[3],x.shape)
# x = qkv[4](x)
print(qkv[4],x.shape)
print(dd_[1].forward)
model.forward

torch.Size([4, 196, 768])
torch.Size([4, 196, 2304])
Identity() torch.Size([4, 196, 2304])
Identity() torch.Size([4, 196, 2304])
Dropout(p=0.0, inplace=False) torch.Size([4, 196, 2304])
Linear(in_features=768, out_features=768, bias=True) torch.Size([4, 196, 2304])
<bound method Attention.forward of Attention(
  (qkv): Linear(in_features=768, out_features=2304, bias=True)
  (q_norm): Identity()
  (k_norm): Identity()
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=768, out_features=768, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)>


<bound method VisionTransformer.forward of VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.

In [12]:
x = torch.arange(27,dtype=torch.float32).reshape(3, 3, 3)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        for k in range(x.shape[2]):
            x[i][j][k] = i

print(x)
nn.LayerNorm((3,3,3))(x)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]]])


tensor([[[-1.2247, -1.2247, -1.2247],
         [-1.2247, -1.2247, -1.2247],
         [-1.2247, -1.2247, -1.2247]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 1.2247,  1.2247,  1.2247],
         [ 1.2247,  1.2247,  1.2247],
         [ 1.2247,  1.2247,  1.2247]]], grad_fn=<NativeLayerNormBackward0>)

In [13]:
dummy_input.shape

torch.Size([4, 196, 768])

In [14]:
blocks = list()
for layer in range(12):
    block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
    ls = [list(block['attn'].children())[0], list(block['attn'].children())[4],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]

    weight_ls = list()
    for i in range(len(ls)):
        weight_ls.append(ls[i].weight.clone().detach().T)
        weight_ls.append(ls[i].bias.clone().detach())
    blocks.append(weight_ls)

    

for i in blocks[0]:
    print(i.shape)
    

torch.Size([768, 2304])
torch.Size([2304])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 3072])
torch.Size([3072])
torch.Size([3072, 768])
torch.Size([768])


In [15]:
torch.round(blocks[0][0][:8, 16:24] * 100)/100

tensor([[-0.0000,  0.0000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0200,  0.0100,  0.0200,  0.0200,  0.0700, -0.0100,  0.0700, -0.1100],
        [ 0.0100,  0.0600, -0.0100, -0.1000,  0.0100,  0.0000,  0.0600, -0.0200],
        [ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0300,  0.0500, -0.0300,  0.0400,  0.0000,  0.0400, -0.0200,  0.0100],
        [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0100, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -0.0000,  0.0000, -0.0000]])

In [16]:
blk1 = blocks[0]
torch.manual_seed(10)
blk1[0] = blk1[0].to(torch.float32)
blk1[1] = blk1[1].to(torch.float32)

dummy_input = torch.randn((4, 196, 768), dtype=torch.float32)
print(torch.matmul(dummy_input,blk1[0])[0][0][1])
dQKV = torch.add(torch.matmul(dummy_input,blk1[0]), blk1[1])
dQKV = dQKV.view(4, 196, 12, 3, 64)
dQKV = torch.einsum('b p h k d -> b h k p d',dQKV)

tensor(-1.1279)


In [17]:
dQKV.shape

torch.Size([4, 12, 3, 196, 64])

4 : batch
12 : Multi head
3 : QKV

In [29]:
Q = dQKV[0][11][0]
K = dQKV[0][11][1]
V = dQKV[0][11][2]

In [32]:
# torch.matmul(torch.nn.functional.softmax(torch.matmul(Q, K.T), dim=1),V)[188:196,8:16]#일반 ATTN
torch.round(torch.matmul(torch.nn.functional.softmax(torch.matmul(Q, K.T), dim=1),V)[188:,64-8:64]*1000)/1000#일반 ATTN

tensor([[-0.1010,  0.0250,  0.1140, -0.0760,  0.2520,  0.0210,  0.1320,  0.0580],
        [-0.1040, -0.0690,  0.0040,  0.0730,  0.1050, -0.0370,  0.0130,  0.0560],
        [-0.1130, -0.0590, -0.1950, -0.0380,  0.0910, -0.0030,  0.1160, -0.0250],
        [-0.1420, -0.0270,  0.0880,  0.0630,  0.0510, -0.0040,  0.1000, -0.0370],
        [-0.0640,  0.0090,  0.2060,  0.0890,  0.1730,  0.1010,  0.0210, -0.0030],
        [-0.1120, -0.0040,  0.0780,  0.0350,  0.2750,  0.0000, -0.0050,  0.0410],
        [-0.0460,  0.1510,  0.0790, -0.0710,  0.0720, -0.1020,  0.1840,  0.2080],
        [-0.2070,  0.0510,  0.0290,  0.0890,  0.1710,  0.0070, -0.0420,  0.0620]])

In [20]:
torch.nn.functional.softmax(torch.arange(25, dtype=torch.float).reshape(5,5),dim=1)

tensor([[0.0117, 0.0317, 0.0861, 0.2341, 0.6364],
        [0.0117, 0.0317, 0.0861, 0.2341, 0.6364],
        [0.0117, 0.0317, 0.0861, 0.2341, 0.6364],
        [0.0117, 0.0317, 0.0861, 0.2341, 0.6364],
        [0.0117, 0.0317, 0.0861, 0.2341, 0.6364]])

In [21]:
tmp = torch.matmul(Q, K.T)[:8,:8]
tmp_emp = torch.exp(tmp - torch.max(tmp, dim=1)[0].view(-1, 1))
torch.round(tmp_emp*100)/100

tensor([[0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0300, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0700, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.1400, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4800, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [22]:
Q = dQKV[0][0][0]
K = dQKV[0][0][1]

V = dQKV[0][0][2]
M = torch.Tensor(8*[-10000000]).view(-1, 1)
L = torch.zeros(8).view(-1,1)
O = torch.zeros((8,64))

for i in range(0, 192, 8):

    tmp = torch.matmul(Q[:8,:], K[i:i+8,:].T)
    tmp_max = torch.maximum(torch.max(tmp, dim=1)[0].view(-1, 1), M)
    P = torch.exp(tmp - tmp_max)
    # print("==K==")
    # for j in K[i:i+8,:4]:
    #     for k in j:
    #         print(round(k.item(),4),end=" ")
    #     print()
    print('\n++++++++++++++',i//8,'+++++++++++++++')
    print("==P==")
    for j in P[:4,:4]:
        for k in j:
            print(round(k.item(),4),end=" ")
        print()
    
    L = L * torch.exp(M-tmp_max) + torch.sum(P, dim=1).view(-1, 1)
    print("==L==")
    for j in L.flatten():
        print(round(j.item(), 2), end=" ")
    print()
    if M[0][0] > -1000:
        O = O*torch.exp(M-tmp_max) + torch.matmul(P, V[i:i+8,:])
    else:
        O = torch.matmul(P, V[i:i+8,:])

    print("==V==")
    for j in V[i:i+8,:4]:
        for k in j:
            print(round(k.item(),4),end=" ")
        print()
    print("==O==")
    for j in O[:4,:4]:
        for k in j:
            print(round(k.item(),2),end=" ")
        print()
    M = tmp_max

O = O/L

torch.round(O[:4,:4])
for i in O[:4,:4]:
    for j in i:
        print(round(j.item(),3),end="\t")
    print()


++++++++++++++ 0 +++++++++++++++
==P==
0.0 1.0 0.0 0.0016 
0.0 0.0335 0.0 1.0 
0.0 1.0 0.0 0.0 
0.0 1.0 0.0 0.0687 
==L==
1.0 1.03 1.0 1.07 1.0 1.14 1.48 1.0 
==V==
-1.0966 0.4385 0.467 0.3408 
-0.3917 0.1036 -0.8208 0.2619 
-1.5464 -0.1394 0.1607 0.5179 
-1.4375 -1.5007 -2.1937 2.7009 
-0.9083 -0.5875 -1.4028 0.1797 
-0.5917 -0.4136 0.2884 -1.2574 
-0.3567 0.127 -1.4185 1.1774 
-0.6361 -0.8299 0.0542 0.0963 
==O==
-0.39 0.1 -0.82 0.27 
-1.45 -1.5 -2.22 2.71 
-0.39 0.1 -0.82 0.26 
-0.49 0.0 -0.97 0.45 

++++++++++++++ 1 +++++++++++++++
==P==
0.0028 0.0 0.0 0.0 
0.0007 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 
0.0014 0.0 0.0 0.0 
==L==
1.26 1.2 1.0 1.07 1.45 1.28 1.57 1.18 
==V==
-0.5827 -1.516 -0.9033 1.7767 
-0.4209 -0.6328 -1.1421 -0.1415 
-0.4936 0.6809 1.4148 0.539 
-1.2153 0.7272 0.5814 -0.6602 
0.0069 -1.2134 -2.8152 1.9007 
-1.3064 -1.0124 -0.1074 -0.0034 
-1.6961 0.5069 1.4507 0.0407 
-0.7878 -1.4801 -1.2407 1.403 
==O==
-0.73 -0.16 -0.86 0.27 
-1.47 -1.69 -2.64 2.99 
-0.39 0.1 -0.82 0.26

In [23]:
# torch.sum(torch.Tensor(7*[8*[-100000000]]), dim=1)
print(M)
torch.maximum(torch.max(torch.arange(64).reshape(8,8), dim=1)[0].view(-1, 1), M)

tensor([[58.0175],
        [41.0226],
        [49.0482],
        [51.3955],
        [50.6254],
        [45.4523],
        [49.4336],
        [58.0317]])


tensor([[58.0175],
        [41.0226],
        [49.0482],
        [51.3955],
        [50.6254],
        [47.0000],
        [55.0000],
        [63.0000]])

([4, 12, 3, 196, 64])

In [24]:
dQKV.reshape(4, 196, 2304)[0][8:16,:8]

tensor([[-0.0171, -0.1672,  0.1841, -1.3738, -1.8454,  0.5362, -1.0144,  0.1519],
        [ 0.8407, -1.9227,  1.2822,  0.7117, -1.0879,  1.1466, -0.5234,  0.6682],
        [ 2.8590, -1.6855,  0.8893,  0.6758, -0.0857,  1.0527, -0.9685, -0.3995],
        [-0.9083, -0.5875, -1.4028,  0.1797,  0.8012, -0.4761,  1.7169, -0.2354],
        [-1.0891,  0.4025,  0.2366, -0.9368,  0.4184,  0.6489,  2.2066,  0.3225],
        [-0.8220, -0.5882, -2.5142,  2.5166,  1.8524, -2.0508,  1.9896, -1.5995],
        [-0.4542,  0.7759, -0.5213,  0.0137,  0.5711, -0.8187,  1.1955,  1.3803],
        [-0.3988, -0.8358, -0.5058,  0.4592, -0.0558, -1.9046,  1.9274, -0.3382]])